In [159]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix


## Reading in the data and data preprocessing

In [2]:
# Reading the data into a data frame
df = pd.read_csv('year_2019.csv')

C:\Users\Emerson\AppData\Local\Temp\ipykernel_9428\736280085.py:1: DtypeWarning: Columns (5,22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('year_2019.csv')


In [57]:
print(df.shape)
df.head(10)

(17573984, 99)


,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2019,549300BX448ALT10FI43,41180,MO,29099.0,29099701000.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,8047,6.15,81200,87,2589,3193,28
1,2019,549300BX448ALT10FI43,39540,WI,55101.0,55101001900.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3908,4.84,69400,116,1310,1509,47
2,2019,549300BX448ALT10FI43,37860,FL,12033.0,12033002604.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4188,19.94,69200,137,1208,1941,24
3,2019,549300BX448ALT10FI43,99999,KY,21161.0,21161960300.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4064,16.26,49800,108,962,1858,49
4,2019,549300BX448ALT10FI43,49340,MA,25027.0,25027730300.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4840,21.12,95300,95,1111,1960,76
5,2019,549300BX448ALT10FI43,49420,WA,53077.0,53077001800.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,7346,61.90,60600,128,1733,2286,37
6,2019,549300BX448ALT10FI43,40900,CA,6067.0,6067006800.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7722,83.42,84000,30,630,1649,57
7,2019,549300BX448ALT10FI43,27140,MS,28089.0,28089030400.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,14840,30.92,60400,170,4468,5134,11
8,2019,549300BX448ALT10FI43,24300,CO,8077.0,8077001101.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7982,16.89,69700,86,2181,3263,29
9,2019,549300BX448ALT10FI43,99999,MN,27049.0,27049080700.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3437,6.84,68800,119,1110,1408,69


In [142]:
# Sampling to 15,000 rows
sampled_df = df.sample(n=15000, random_state=2023)

In [143]:
# Dropping denial reason because it perfectly predicts approval or denial 
sampled_df = sampled_df.drop(columns=['denial_reason-1', 'denial_reason-2', 'denial_reason-3', 'denial_reason-4'])

In [144]:
# Getting NA counts for each column
na_counts = sampled_df.isna().sum()

# Compute the percentage of NAs for each column
na_percentage = (na_counts / len(sampled_df)) * 100

# Create a sorted list with column name, NA count, and NA percentage
sorted_columns_with_na = sorted(zip(sampled_df.columns, na_counts, na_percentage), key=lambda x: x[1], reverse=True)

sorted_columns_with_na


[('applicant_ethnicity-4', 15000, 100.0),
 ('applicant_ethnicity-5', 15000, 100.0),
 ('co-applicant_ethnicity-4', 15000, 100.0),
 ('co-applicant_ethnicity-5', 15000, 100.0),
 ('applicant_race-5', 14999, 99.99333333333334),
 ('co-applicant_race-5', 14999, 99.99333333333334),
 ('co-applicant_ethnicity-3', 14997, 99.98),
 ('co-applicant_race-4', 14995, 99.96666666666667),
 ('applicant_race-4', 14994, 99.96000000000001),
 ('applicant_ethnicity-3', 14993, 99.95333333333333),
 ('co-applicant_race-3', 14977, 99.84666666666666),
 ('applicant_race-3', 14942, 99.61333333333333),
 ('aus-5', 14888, 99.25333333333334),
 ('aus-4', 14863, 99.08666666666667),
 ('co-applicant_ethnicity-2', 14781, 98.54),
 ('co-applicant_race-2', 14766, 98.44000000000001),
 ('aus-3', 14701, 98.00666666666666),
 ('multifamily_affordable_units', 14579, 97.19333333333333),
 ('total_points_and_fees', 14530, 96.86666666666667),
 ('applicant_race-2', 14407, 96.04666666666667),
 ('applicant_ethnicity-2', 14370, 95.8),
 ('aus-2

In [145]:
# Define the percentage threshold to drop columns
threshold_percentage = 90

# Find the columns to drop
columns_to_drop = [col for col, count, percent in sorted_columns_with_na if percent > threshold_percentage]

print(len(columns_to_drop))
print(columns_to_drop)

# Dropping the columns from the DataFrame that have too many NA values
sampled_df.drop(columns=columns_to_drop, inplace=True)

23
['applicant_ethnicity-4', 'applicant_ethnicity-5', 'co-applicant_ethnicity-4', 'co-applicant_ethnicity-5', 'applicant_race-5', 'co-applicant_race-5', 'co-applicant_ethnicity-3', 'co-applicant_race-4', 'applicant_race-4', 'applicant_ethnicity-3', 'co-applicant_race-3', 'applicant_race-3', 'aus-5', 'aus-4', 'co-applicant_ethnicity-2', 'co-applicant_race-2', 'aus-3', 'multifamily_affordable_units', 'total_points_and_fees', 'applicant_race-2', 'applicant_ethnicity-2', 'aus-2', 'prepayment_penalty_term']


In [146]:
for col in sampled_df.columns:
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(sampled_df[col]):
        # Fill NA with mean
        sampled_df[col].fillna(sampled_df[col].mean(), inplace=True)
    else:
        # Fill NA with mode
        sampled_df[col].fillna(sampled_df[col].mode().iloc[0], inplace=True)


In [147]:
print(sampled_df.shape)
sampled_df.isna().sum().sum()

(15000, 72)


0

#### Some numeric variables are being stored as categorical

In [148]:
# Figuring out which columns have too many categories

# Get the categorical columns (those with the 'object' or 'category' data type)
categorical_columns = sampled_df.select_dtypes(include=['object', 'category']).columns

# Create a dictionary to store the number of unique categories for each categorical variable
category_counts = {}

# Iterate through the categorical columns and count the unique values
for col in categorical_columns:
    category_counts[col] = sampled_df[col].nunique()


sorted_category_counts = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)


sorted_category_counts



[('census_tract', 12340),
 ('total_loan_costs', 7396),
 ('loan_to_value_ratio', 4989),
 ('rate_spread', 4491),
 ('origination_charges', 4086),
 ('discount_points', 2580),
 ('lender_credits', 2396),
 ('lei', 1887),
 ('interest_rate', 652),
 ('property_value', 436),
 ('loan_term', 178),
 ('intro_rate_period', 60),
 ('state_code', 52),
 ('debt_to_income_ratio', 28),
 ('total_units', 13),
 ('co-applicant_age', 11),
 ('applicant_age', 10),
 ('derived_race', 9),
 ('derived_loan_product_type', 7),
 ('derived_ethnicity', 5),
 ('derived_dwelling_category', 4),
 ('derived_sex', 4),
 ('conforming_loan_limit', 3),
 ('applicant_age_above_62', 2),
 ('co-applicant_age_above_62', 2)]

##### Finding which columns have 'Exempt' values

In [150]:
columns_with_exempt = []

# Iterate through all columns
for col in sampled_df.columns:
    # Check if 'Exempt' is in the column
    if (sampled_df[col] == 'Exempt').any():
        columns_with_exempt.append(col)


print(columns_with_exempt)

print(sampled_df.shape)
print(sampled_df.isna().sum().sum())

['loan_to_value_ratio', 'interest_rate', 'rate_spread', 'total_loan_costs', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'intro_rate_period', 'property_value', 'debt_to_income_ratio']
(15000, 72)


0

##### Replacing 'Exempt' with an arbitrarily large number. You can do this in decision trees. 

In [151]:
# Iterate through the specified columns
for col in columns_with_exempt:
    # Replace the word 'Exempt' with the number 9999999
    sampled_df[col] = sampled_df[col].replace('Exempt', '9999999')
    
    # Convert the column to a numeric data type
    sampled_df[col] = pd.to_numeric(sampled_df[col], errors='coerce')

print(sampled_df.shape)
print(sampled_df.isna().sum().sum())

(15000, 72)
10387


In [153]:
# Figuring out which columns have too many categories

# Get the categorical columns (those with the 'object' or 'category' data type)
categorical_columns = sampled_df.select_dtypes(include=['object', 'category']).columns

# Create a dictionary to store the number of unique categories for each categorical variable
category_counts = {}

# Iterate through the categorical columns and count the unique values
for col in categorical_columns:
    category_counts[col] = sampled_df[col].nunique()


sorted_category_counts = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)

sorted_category_counts

[('census_tract', 12340),
 ('lei', 1887),
 ('state_code', 52),
 ('total_units', 13),
 ('co-applicant_age', 11),
 ('applicant_age', 10),
 ('derived_race', 9),
 ('derived_loan_product_type', 7),
 ('derived_ethnicity', 5),
 ('derived_dwelling_category', 4),
 ('derived_sex', 4),
 ('conforming_loan_limit', 3),
 ('applicant_age_above_62', 2),
 ('co-applicant_age_above_62', 2)]

In [154]:
# Dropping census tract and lei
sampled_df.drop(columns=['census_tract', 'lei'], inplace=True)

In [166]:
# Replacing NA values

for col in sampled_df.columns:
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(sampled_df[col]):
        # Fill NA with mean
        sampled_df[col].fillna(sampled_df[col].mean(), inplace=True)
    else:
        # Fill NA with mode
        sampled_df[col].fillna(sampled_df[col].mode().iloc[0], inplace=True)

print(sampled_df.shape)
print(sampled_df.isna().sum().sum())

(15000, 70)
0


In [156]:
# Identify the categorical columns (optional, if you know the names already)
categorical_columns = sampled_df.select_dtypes(include=['object', 'category']).columns

# Convert categorical columns to dummy variables
sampled_df_dummies = pd.get_dummies(sampled_df, columns=categorical_columns)

print(sampled_df_dummies.shape)
sampled_df_dummies.isna().sum().sum()

(15000, 180)


0

## Model build and fit

In [157]:
X = sampled_df_dummies.drop(columns=['action_taken']) 
y = sampled_df_dummies['action_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [158]:
rf0 = RandomForestClassifier(n_estimators=100, random_state=42)
rf0.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

### Confusion Matrix

In [160]:
y_pred = rf0.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1578    2    2    0    0    1    0    0]
 [  11   47    3    1    0    0    0    1]
 [   8    0  407    7    0    0    0    0]
 [   3    0    0  353    6    0    0    0]
 [   0    0    0  136   14    0    0    0]
 [   1    0    0    0    0  392    0    0]
 [   0    0    0    0    0    0   13    1]
 [   0    1    0    0    0    0    1   11]]


In [161]:
importances = rf0.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
sorted_features = [X.columns[idx] for idx in sorted_indices]
sorted_features

['rate_spread',
 'hoepa_status',
 'applicant_credit_score_type',
 'property_value',
 'total_loan_costs',
 'submission_of_application',
 'interest_rate',
 'initially_payable_to_institution',
 'origination_charges',
 'purchaser_type',
 'co-applicant_credit_score_type',
 'loan_to_value_ratio',
 'debt_to_income_ratio',
 'aus-1',
 'applicant_race-1',
 'applicant_sex',
 'applicant_ethnicity-1',
 'applicant_age_8888',
 'income',
 'loan_amount',
 'tract_population',
 'tract_owner_occupied_units',
 'tract_to_msa_income_percentage',
 'tract_minority_population_percent',
 'tract_one_to_four_family_homes',
 'ffiec_msa_md_median_family_income',
 'derived_msa-md',
 'lender_credits',
 'tract_median_age_of_housing_units',
 'county_code',
 'co-applicant_age_8888',
 'occupancy_type',
 'open-end_line_of_credit',
 'derived_sex_Sex Not Available',
 'loan_purpose',
 'co-applicant_ethnicity-1',
 'applicant_ethnicity_observed',
 'loan_term',
 'co-applicant_race-1',
 'business_or_commercial_purpose',
 'applica

In [168]:
sorted_features_with_importance = []
for i, idx in enumerate(sorted_indices):
    feature_name = X.columns[idx]
    importance_value = importances[idx]
    print(f"{i + 1}. {feature_name} ({importance_value})")
    sorted_features_with_importance.append((feature_name, importance_value))

sorted_features_with_importance

1. rate_spread (0.11481709925248565)
2. hoepa_status (0.08518408458103029)
3. applicant_credit_score_type (0.07236460075622082)
4. property_value (0.0648572876445395)
5. total_loan_costs (0.05846186728252367)
6. submission_of_application (0.053395624669270954)
7. interest_rate (0.052753677680160864)
8. initially_payable_to_institution (0.03687882593284539)
9. origination_charges (0.028389501207910714)
10. purchaser_type (0.028246227159940438)
11. co-applicant_credit_score_type (0.027850418636741318)
12. loan_to_value_ratio (0.01987503941155381)
13. debt_to_income_ratio (0.01463423149408162)
14. aus-1 (0.013609804852630566)
15. applicant_race-1 (0.013468740743674212)
16. applicant_sex (0.012934819141317798)
17. applicant_ethnicity-1 (0.011479938256111323)
18. applicant_age_8888 (0.011175641705423773)
19. income (0.010596802439396182)
20. loan_amount (0.009922164387804643)
21. tract_population (0.009167989766113258)
22. tract_owner_occupied_units (0.008959549628288814)
23. tract_to_msa_i

[('rate_spread', 0.11481709925248565),
 ('hoepa_status', 0.08518408458103029),
 ('applicant_credit_score_type', 0.07236460075622082),
 ('property_value', 0.0648572876445395),
 ('total_loan_costs', 0.05846186728252367),
 ('submission_of_application', 0.053395624669270954),
 ('interest_rate', 0.052753677680160864),
 ('initially_payable_to_institution', 0.03687882593284539),
 ('origination_charges', 0.028389501207910714),
 ('purchaser_type', 0.028246227159940438),
 ('co-applicant_credit_score_type', 0.027850418636741318),
 ('loan_to_value_ratio', 0.01987503941155381),
 ('debt_to_income_ratio', 0.01463423149408162),
 ('aus-1', 0.013609804852630566),
 ('applicant_race-1', 0.013468740743674212),
 ('applicant_sex', 0.012934819141317798),
 ('applicant_ethnicity-1', 0.011479938256111323),
 ('applicant_age_8888', 0.011175641705423773),
 ('income', 0.010596802439396182),
 ('loan_amount', 0.009922164387804643),
 ('tract_population', 0.009167989766113258),
 ('tract_owner_occupied_units', 0.00895954